# Preliminaries

In [1]:
!pip install gpytorch
  
import math
import torch
import gpytorch 
import sklearn
from sklearn.feature_selection import mutual_info_regression
from matplotlib import pyplot as plt
from functools import reduce

import torch.nn as nn
import torch.nn.functional as F
from torch.autograd import Variable


%matplotlib inline
%load_ext autoreload
%autoreload 2



     |████████████████████████████████| 215kB 2.8MB/s 
  Created wheel for gpytorch: filename=gpytorch-0.3.5-py2.py3-none-any.whl size=349719 sha256=c5a5cd99a1a9e2b3aabc2ba9b9e6b047848a1acc5180801777e88abeb9d0ddd2
  Stored in directory: /root/.cache/pip/wheels/d6/31/88/c43a94e0073a54056ac663366f2195de36535b38a81a378196
Successfully built gpytorch


In [2]:
from google.colab import drive
drive.mount('/content/gdrive')
%cd gdrive/My Drive/MSC Thesis

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/gdrive
/content/gdrive/My Drive/MSC Thesis


In [0]:
import time
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import datetime

predictors = pd.read_csv('pred_inputs.csv')
predict_these = np.unique(predictors['INSTRUMENT'])

def from_csv(instrument):
    df = pd.read_csv('Data/' + instrument + '.csv')
    df['date'] = pd.to_datetime(df['date']).dt.date
    df.set_index('date',inplace=True)
    df = df.sort_values(by = 'date')
    df.columns = [instrument]
    
    return df
  
def preprocessing(commodity):
    #####################################################################################
    # This ensures chronological order (some time series are NOT in order)              #
    #####################################################################################
    
    prices_df = db.get_instrument_data(commodity)
    prices_df = prices_df.sort_values(by = 'date')
    prices_df.columns = [commodity]
    return prices_df
  
def get_predictors(commodity, predictors):
    ####################################################################
    # This returns the df containing the values of the commodity to be #
    # predicted as well as that of its predictors.                     #
    #                                                                  #
    # Args:                                                            #
    # 1) commodity: name of the commodity                              #
    # 2) predictors: the dataframe sent by Michael Button              #
    ####################################################################
    
    # get a list of all its predictors
    Pred = list(predictors.loc[predictors['INSTRUMENT'] == commodity]['INPUT'])
    Pred = [p for p in Pred if p != commodity]
    
    # Initialize the dataframe with the commodity we want to predict
    DF = from_csv( commodity )
    
    # Keep adding the predictors
    for predictor in Pred:
        temp_df = from_csv( predictor )
        DF = DF.join(temp_df)
        
    DF.index = pd.to_datetime(DF.index)
    return DF
  
def price_to_returns(prices_df, diff):
    ###############################################################
    # This changes the price data to 5 day log difference.        #
    # If X has negative values, use the following scheme:         # 
    #          X := X - min(X) + 1                                #
    ###############################################################
    
    # forward fill
    prices_df.fillna(method='ffill', inplace = True)
    
    returns_df = prices_df.copy()
    for colname in prices_df.columns:
        temp = prices_df.loc[:,colname]
        
        if np.min(temp) <= 0:
            temp = temp - np.min(temp) + 1
        returns_df[colname] = np.log(temp) - np.log(temp.shift(diff))

    # drop rows with NaN
    returns_df.dropna(inplace = True)
    return returns_df


  
def feature_generation(metal, time_pred = [1,5,22]):
  metal = metal + '_lme_prices'
  Metal_DF = get_predictors(metal, predictors)
  # remove comex, since comex is not 
  Metal_DF = Metal_DF[[col for col in list(Metal_DF.columns) if 'comex' not in col]]
  
  # 1, 5, 22 days
  # log difference L
  LD = [price_to_returns(Metal_DF, t_pred) for t_pred in time_pred]

  # EWMA of L
  # EWMA = [L.ewm(span = horizon).mean() for horizon, L in zip(time_pred, LD)]
  EWMA = [LD[0].ewm(halflife = horizon).mean() for horizon in time_pred ]

  # EWMV of L
  # 1. calculate expanding window mean for the returns (1 day) 
  # 2. subtract rolling mean and take square
  EM = LD[0].expanding(2).mean() 
  EWMV = [((LD[0] - EM)**2).ewm(span = horizon).mean()**0.5 for horizon in time_pred ]
  
  # rename columns
  for horizon, ld, ewma, ewmv in zip(time_pred, LD, EWMA, EWMV):
    ld.columns = [col + '_LD_' + str(horizon) for col in ld.columns]
    ewma.columns = [col + '_EWMA_' + str(horizon) for col in ewma.columns]
    ewmv.columns = [col + '_EWMV_' + str(horizon) for col in ewmv.columns]
    
  # merge together 
  ALL_FEATURES = pd.concat([pd.concat(DFS, axis = 1, sort=True) for DFS in [LD, EWMA, EWMV]], axis = 1)
  ALL_FEATURES_columns = list(ALL_FEATURES.columns)
  ALL_FEATURES_columns.sort()
  ALL_FEATURES = ALL_FEATURES.loc[:,ALL_FEATURES_columns]
  ALL_FEATURES.dropna(inplace = True)
  
  return ALL_FEATURES



def feature_extraction(DF, column_name, lag):
  # This extracts the relevant predictors for the column_name
  # threshold is the % of predictors that we want to include
  # Mutual Information
  DF_metal = DF.loc[:,[column_name]]
  MI_table = np.zeros(DF.shape[1])
  for i in range(DF.shape[1]):
      MI = sklearn.feature_selection.mutual_info_regression(DF_metal, DF.iloc[:,i])
      MI_table[i] = MI
  
  tol = 1e-3
  MI_table_2 = MI_table[MI_table > tol]
  out, bins = pd.qcut(MI_table_2, [0.9, 1], retbins=True)
  selected_cols = list(MI_table > bins[-2])
  
  DF = DF.loc[:,selected_cols].shift(periods = lag)
  DF.columns = [col + '_lag' for col in DF.columns]
  DF = pd.concat([DF_metal, DF], axis = 1, sort = True).dropna()
  time_column = pd.DataFrame(index = DF.index, 
                             data = np.arange(DF.shape[0])*0.01 , 
                             columns = ['time'] )
  DF = pd.concat([DF, time_column], axis = 1)
  # add time column
  
  
  return(DF)



# 1. Balanced Decision Standard Multitask GP

## 1.1 Some helper functions

In [0]:
def kron(X, Y):
    Y_ = Y.repeat(X.shape[0],X.shape[1])
    y1,y2 = Y.shape
    x1,x2 = X.shape
    
    X_ = X.flatten()[:,None].repeat(1, y2).reshape(x1, x2*y2).repeat(1,y1).reshape(x1*y1, x2*y2)
    
    return(X_*Y_)

def softplus(param):
    return torch.log(1 + torch.exp(param) )    
    
class RBF:
    def __init__(self, dimensions, ARD = True):
        self.sigma = torch.nn.Parameter(torch.randn(dimensions)) 
        self.params = [self.sigma]
    
    def forward(self, X1, X2):
        if len(X1.shape) == 1:
            X1 = torch.unsqueeze(X1, 0)
            X2 = torch.unsqueeze(X2, 1)
            return torch.exp( -((X1 - X2)/softplus(self.sigma) )**2 )
        else:
            return torch.exp(-torch.norm((X1[:,None] - X2[None,:])/softplus(self.sigma), dim = 2)**2)

class MGP:
    def __init__(self, X, Y, kernel, rank, ARD):
        # X has size N x DIM
        # Y has size N X tasks
        self.X = X
        if len(self.X.shape) == 1:
            self.X = self.X[:,None]
        self.n_tasks = Y.shape[1]
        self.n_datapoints = Y.shape[0]
        
        self.Y = Y.t().flatten()
        self.kernel = kernel(X.shape[1], ARD)
        
        # initialize parameters
        self.noise = torch.nn.Parameter(torch.randn(self.n_tasks) ) 
        assert rank <= self.n_tasks
        # cov = task_cov task_cov^T
        self.task_cov = torch.nn.Parameter(torch.randn(self.n_tasks,rank) ) 
        
    def get_params(self):
        return self.kernel.params + [self.task_cov, self.noise]
    
    def get_cov(self):

        K = kron(self.task_cov.mm(self.task_cov.t()), self.kernel.forward(self.X, self.X))  \
            + kron(torch.diag(softplus(self.noise)), torch.eye(self.n_datapoints))
        return K
    
    def get_K_xX(self, x):
        if len(x.shape) == 1:
            x = x[:,None]
        K_xX = kron(self.task_cov.mm(self.task_cov.t()), self.kernel.forward(x, self.X))
        return K_xX
    
    def get_K_xx(self, x):
        if len(x.shape) == 1:
            x = x[:,None]
        K_xx = kron(self.task_cov.mm(self.task_cov.t()), self.kernel.forward(x, x)) \
                + kron(torch.diag(softplus(self.noise)), torch.eye(x.shape[0]))
        return K_xx

class SIGP: #simultaneous independent GP
    def __init__(self, X, Y, kernel, ARD):
        # X has size N x DIM
        # Y has size N X tasks
        self.X = X
        if len(self.X.shape) == 1:
            self.X = self.X[:,None]
        self.n_tasks = Y.shape[1]
        self.n_datapoints = Y.shape[0]
        self.Y = Y.t().flatten()
        self.kernels = [kernel(X.shape[1], ARD) for i in range(self.n_tasks)]
        
        # initialize parameters
        self.noise = torch.nn.Parameter(torch.randn(self.n_tasks) ) 

    def get_params(self):
        return reduce(lambda x, y: x+y, 
                      [ker.params for ker in self.kernels]) + [self.noise]
    
    def get_cov(self):
        
        list_kernels = [k.forward(self.X, self.X) for k in self.kernels]
        BD = Variable(torch.zeros(self.n_tasks * self.n_datapoints, 
                                  self.n_tasks * self.n_datapoints))
        for k, ker in enumerate(list_kernels):
            BD[ k * self.n_datapoints: (k+1) * self.n_datapoints,
                k * self.n_datapoints: (k+1) * self.n_datapoints] = ker
        
        K = BD + kron(torch.diag(softplus(self.noise)), torch.eye(self.n_datapoints))
        return K
    
    def get_K_xX(self, x):
        if len(x.shape) == 1:
            x = x[:,None]
        list_kernels = [k.forward(x, self.X) for k in self.kernels]
        K_xX = Variable(torch.zeros(self.n_tasks * x.shape[0] , self.n_tasks * self.n_datapoints))
        for k, ker in enumerate(list_kernels):
            K_xX[ k * x.shape[0]: (k+1) * x.shape[0],
                k * self.n_datapoints: (k+1) * self.n_datapoints] = ker
        return K_xX
    
    def get_K_xx(self, x):
        if len(x.shape) == 1:
            x = x[:,None]
        list_kernels = [k.forward(x, x) for k in self.kernels]
        K_xx = Variable(torch.zeros(self.n_tasks * x.shape[0] , self.n_tasks * x.shape[0]))
        for k, ker in enumerate(list_kernels):
            K_xx[ k * x.shape[0]: (k+1) * x.shape[0],
                  k * x.shape[0]: (k+1) * x.shape[0]] = ker
        return K_xx

    
    
class Warp_MT:
    def __init__(self, tasks, complexity):
        self.Alphas = torch.nn.Parameter(torch.randn(1,complexity, tasks))
        self.B = torch.nn.Parameter(torch.randn(complexity,1,1))
        self.C = torch.nn.Parameter(torch.randn(complexity,1,1))
        self.params = [self.Alphas, self.B, self.C]
        
    def f(self, Y):
        # B : k x 1 vector
        # C : k x 1 vector
        # Alphas : k x m matrix
        # Y : n x m matrix (n examples, m tasks)

        # returns a n x m matrix of warpings. 

        Y = Y[None, : ]

        temp = torch.tanh(softplus(self.B)* Y + self.C)
        temp = torch.einsum('ijk, jlk -> ilk', softplus(self.Alphas), temp) + Y
        return(temp)
    
    def df(self, Y):

        Y = Y[None, : ]

        temp = (1 - torch.tanh(softplus(self.B)* Y + self.C)**2 ) * softplus(self.B)
        temp = torch.einsum('ijk, jlk -> ilk', softplus(self.Alphas), temp) + 1
        return(temp)
    
    def get_params(self):
        return self.params
    
    
    def inv(self, eps, Z):
        Z = Z[None,:]

        tol = 1
        n = 0
        while tol > eps and n < 10:
            n += 1
            Y = torch.zeros(Z.shape).squeeze()
            for i in range(10):
                _f = (Z - self.f(Y) ).squeeze()
                _df = self.df(Y).squeeze()
                Y +=  _f/_df 
            Z_1 = self.f(Y).squeeze().reshape(-1,2)
            
            tol = torch.max(torch.abs((Z_1 - Z.squeeze() )/ Z.squeeze() ) )
            
            tol_flag = torch.max(torch.abs((Z_1 - Z.squeeze() )/ Z.squeeze() ), dim = 1 ).values < eps
                                 
        return(tol_flag,Y)


  
class WCSMGP_Likelihood:
    def __init__(self):
        pass
    
    def forward(self, K, Y, Warp):
        Cholesky = torch.cholesky(K)
        Z = Warp.f(Y).squeeze().t().flatten()

        Likelihood = 0.5*2*torch.sum(torch.log(torch.diag(Cholesky))) \
                + 0.5* Z.unsqueeze(0).mm( torch.inverse(K) ).mm(Z.unsqueeze(1)) \
                - torch.sum(torch.log(Warp.df(Y) ))

        return Likelihood
    
def WCSMGP_predict(X, Y, x, Models, Warp, alphas = None):
    
    # Y should be a n x task matrix
    _Z = Warp.f(Y).squeeze().t().contiguous().reshape(1,-1).t().contiguous()
    
    if alphas == None:
        alphas = torch.ones(len(Models), dtype = torch.float32)
    K_XX = reduce(lambda x, y: x + y, 
                  [alpha*model.get_cov() for alpha,model in zip(alphas, Models)] )
    K_xX = reduce(lambda x, y: x + y, 
                  [alpha*model.get_K_xX(x) for alpha,model in zip(alphas, Models)] )
    K_xx = reduce(lambda x, y: x + y, 
                  [alpha*model.get_K_xx(x) for alpha,model in zip(alphas, Models)] )
    pred_mean = K_xX.mm(torch.inverse(K_XX)).mm(_Z).squeeze()
    
    pred_mean = pred_mean.reshape(2,-1).t().contiguous()
    
    tol_flag, pred_mean = Warp.inv(Z = pred_mean,eps=0.001)
    
    return tol_flag, pred_mean
#     pred_cov = K_xx - K_xX.mm(torch.inverse(K_XX)).mm(K_xX.t())
    
#     return(pred_mean, pred_cov)


def WCSMGP_simulation(complexity, X, Y, x, y):
    #### setup ##########################################
    llh = []
    model_MGP = MGP(X = X, Y= Y, kernel = RBF, rank = 2, ARD = False)
    model_SIGP = SIGP(X = X, Y= Y, kernel = RBF, ARD = False)

    Warp = Warp_MT(tasks = 2, complexity=complexity)
    LLH = WCSMGP_Likelihood()

    alpha = torch.nn.Parameter(torch.tensor(0.))
    optimizer1 = torch.optim.Adam(params = model_MGP.get_params() + \
                                 model_SIGP.get_params() + \
                                 Warp.get_params() + \
                                 [alpha], lr = 0.1)

    #### training ##########################################
    for i in range(1000):

        _alpha = torch.sigmoid(alpha)

        K = _alpha*model_SIGP.get_cov() \
            + (1-_alpha)*model_MGP.get_cov()
        loss = LLH.forward(K, Y, Warp)

        optimizer1.zero_grad()
        loss.backward()
        optimizer1.step()
        
        llh.append(loss.squeeze().detach().numpy())
    
    tol_flag, pred_mean = WCSMGP_predict(X = model_SIGP.X, Y = Y,
                                          Models=[model_SIGP, model_MGP], 
                                          alphas=[_alpha, 1- _alpha],
                                          Warp= Warp,
                                          x=x)
    pred_mean = pred_mean.reshape(-1,2)
    y = y.reshape(-1,2)
#     loss_0 = torch.mean((y[tol_flag,0] - pred_mean[tol_flag,0])**2).detach().numpy()
#     loss_1 = torch.mean((y[tol_flag,1] - pred_mean[tol_flag,1])**2).detach().numpy()
    loss = torch.mean((y[tol_flag,:] - pred_mean[tol_flag,:])**2, 0).detach().numpy()
    
#     sign_0 = torch.mean(torch.sign(y[tol_flag,0]) == torch.sign(pred_mean[tol_flag,0])).detach().numpy()
#     sign_1 = torch.mean(torch.sign(y[tol_flag,1]) == torch.sign(pred_mean[tol_flag,1])).detach().numpy()
    sign = np.mean((torch.sign(y[tol_flag,:]) == torch.sign(pred_mean[tol_flag,:])).detach().numpy(), 0 )
  
    return loss, sign, llh
  
  

  
class Likelihood:
    def __init__(self):
        pass
    
    def forward(self, K, Y):
        Cholesky = torch.cholesky(K)
        Likelihood = 0.5*2*torch.sum(torch.log(torch.diag(Cholesky))) \
            + 0.5* (Y[None,:]).mm(torch.inverse(K) ).mm(Y[:,None]) 
        return Likelihood

    
def predict(X, Y, x, Models, alphas = None):
    if alphas == None:
        alphas = torch.ones(len(Models), dtype = torch.float32)
    K_XX = reduce(lambda x, y: x + y, 
                  [alpha*model.get_cov() for alpha,model in zip(alphas, Models)] )
    K_xX = reduce(lambda x, y: x + y, 
                  [alpha*model.get_K_xX(x) for alpha,model in zip(alphas, Models)] )
    K_xx = reduce(lambda x, y: x + y, 
                  [alpha*model.get_K_xx(x) for alpha,model in zip(alphas, Models)] )
    pred_mean = K_xX.mm(torch.inverse(K_XX)).mm(Y[:,None]).squeeze().reshape(2,-1).t().contiguous()
    pred_cov = K_xx - K_xX.mm(torch.inverse(K_XX)).mm(K_xX.t())
    
    return(pred_mean, pred_cov)


def CSMGP_simulation(X, Y, x, y):
    llh = []
    model_MGP = MGP(X = X, Y= Y, kernel = RBF, rank = 2, ARD = False)
    model_SIGP = SIGP(X = X, Y= Y, kernel = RBF, ARD = False)

    LLH = Likelihood()

    alpha = torch.nn.Parameter(torch.tensor(0.))
    optimizer1 = torch.optim.Adam(params = model_MGP.get_params() + \
                                 model_SIGP.get_params() + \
                                 [alpha], lr = 0.1)

    #### training ##########################################
    for i in range(200):

        _alpha = torch.sigmoid(alpha)

        K = _alpha*model_SIGP.get_cov() \
            + (1-_alpha)*model_MGP.get_cov()
        loss = LLH.forward(K, model_SIGP.Y)

        optimizer1.zero_grad()
        loss.backward()
        optimizer1.step()

    
    pred_mean, pred_cov = predict(X = model_SIGP.X, Y = model_SIGP.Y,
                                          Models=[model_SIGP, model_MGP], 
                                          alphas=[_alpha, 1- _alpha],
                                          x=x)
#     loss_0 = torch.mean((y[:,0] - pred_mean[:,0])**2).detach().numpy()
#     loss_1 = torch.mean((y[:,1] - pred_mean[:,1])**2).detach().numpy()
    
    pred_mean = pred_mean.reshape(-1,2)
    y = y.reshape(-1,2)
    
    loss = torch.mean((y - pred_mean)**2, 0).detach().numpy()
    sign = np.mean((torch.sign(y) == torch.sign(pred_mean)).detach().numpy(), 0)
    
    
    return loss, sign

# 3. Commodities Dataset

In [0]:
def Across_Metals(window, horizon, Target_Variables, trials, complexity):

  # metal is either 'al' or 'cu'
  # window is the length of data before prediction
  # Horizons is a list of numbers (e.g. [1,5,22])
  # Target_Variables is a list of strings 
  #      (e.g. ['al_lme_prices_LD_1', 'cu_lme_prices_LD_1'])
  # kernel is the kernel that we will use 
  # trials (self explanatory)
  # n_test (number of test points per trial)
  
  I = len(Target_Variables)
  
  # this
  CSMGP_sign = np.zeros((I,trials))
  CSMGP_mse = np.zeros((I,trials))
  WCSMGP_sign = np.zeros((I,len(complexity),trials))
  WCSMGP_mse = np.zeros((I,len(complexity),trials))

  BM_mse = np.zeros((I,trials))
  
  # this contains the list of dataframes 
  DF_list = []
  for TV_index,TV in enumerate(Target_Variables):
    # horizon and target variable
    DF = feature_generation(TV[0:2], time_pred = [1,5,22])
    DF = feature_extraction(DF, TV, horizon)
    DF_list.append(DF)
  
  # now join the DFs in DF_list, and remove duplicate columns
  DF = pd.concat(DF_list, axis = 1, sort = True).dropna()
  DF = DF.loc[:,~DF.columns.duplicated()]
  Y = DF.loc[:,Target_Variables]
  X = DF.drop(Target_Variables, axis = 1)

#   print((Y**2).mean())
  
  no_conv = []
  
  for i in range(trials):
    print(i)
    START = int(np.random.choice(np.arange(len(Y) - 200),1) )
    END = START + window
    X_train = torch.tensor( X.iloc[START:END, :].values , dtype = torch.float32)
    X_test = torch.tensor( X.iloc[[(END+horizon-1)], :].values , dtype = torch.float32)
    Y_train = torch.tensor( Y.iloc[START:END, :].values.squeeze() , dtype = torch.float32)
    Y_test = torch.tensor( Y.iloc[[(END+horizon-1)], :].values.squeeze() , dtype = torch.float32)
    for j, comp in enumerate(complexity):
      try:
        WCSMGP_mse[:,j,i], WCSMGP_sign[:,j,i], llh = WCSMGP_simulation(complexity = comp, 
                                                                X = X_train, Y = Y_train, 
                                                                x = X_test, y = Y_test)
        skip = 0
      except:
        print('bad!', j)
        skip = 1
        break
    if skip == 1:
      skip = 0
      no_conv.append(i)
      continue
    CSMGP_mse[:,i], CSMGP_sign[:,i] = CSMGP_simulation(X = X_train, Y = Y_train, 
                                                       x = X_test, y = Y_test)
    BM_mse[:,i] = Y_test**2
  
  
  return_list = [WCSMGP_mse, WCSMGP_sign, CSMGP_mse, CSMGP_sign, BM_mse]
  
  return_list = [np.delete(elt, no_conv, -1) for elt in return_list]
  return_list = [np.delete(elt, np.where(np.isnan(elt))[-1], -1) for elt in return_list]
  
#   WCSMGP_mse = np.delete(WCSMGP_mse, no_conv, -1)
#   WCSMGP_sign = np.delete(WCSMGP_sign, no_conv, -1)
#   CSMGP_mse = np.delete(CSMGP_mse, no_conv, -1)
#   CSMGP_sign = np.delete(CSMGP_sign, no_conv, -1)
#   BM_mse = np.delete(BM_mse, no_conv, -1)
      
  return return_list#WCSMGP_mse, WCSMGP_sign, CSMGP_mse, CSMGP_sign, BM_mse




In [0]:
Joint_Predictions = [['al_lme_prices_LD_1', 'cu_lme_prices_LD_1'],
                     ['al_lme_prices_LD_5', 'cu_lme_prices_LD_5'],
                     ['al_lme_prices_LD_22', 'cu_lme_prices_LD_22'] ]

horizons = [1, 5, 22]
complexity = [3,6,9]

for h, horizon in enumerate(horizons):
  Target_Variables = Joint_Predictions[h]
  WCSMGP_mse, WCSMGP_sign, CSMGP_mse, CSMGP_sign, BM_mse= Across_Metals(window = 100, 
                                                                     horizon = horizon, 
                                                                     Target_Variables = Target_Variables, 
                                                                     trials = 5,
                                                                     complexity = complexity)
  print('T+',horizon, ' Total Count = ' + str(BM_mse.shape[-1]) )
  for tv, TV in enumerate(Target_Variables):
    print(TV[0:2].upper(),':') #this is the metal
    
    for c, comp in enumerate(complexity):
      print('WCSMGP '+str(comp)+ ' MSE', np.mean(WCSMGP_mse[tv,c,:] ) )
      print('WCSMGP '+str(comp)+ ' sign', np.mean(WCSMGP_sign[tv,c,:]) )
      print(' ')
    
    print('CSMGP MSE', np.mean(CSMGP_mse[tv,:] ))
    print('CSMGP sign', np.mean(CSMGP_sign[tv,:] ))
    print(' ')
          
    print('Benchmark', np.mean(BM_mse[tv,:] ) )
    print(' ')
    
  print('######')

0


/usr/local/lib/python3.6/dist-packages/numpy/core/fromnumeric.py:3118: RuntimeWarning: Mean of empty slice.
  out=out, **kwargs)
/usr/local/lib/python3.6/dist-packages/numpy/core/_methods.py:78: RuntimeWarning: invalid value encountered in true_divide
  ret, rcount, out=ret, casting='unsafe', subok=False)


1
2
3
4
T+ 1  Total Count = 5
AL :
WCSMGP 3 MSE 0.0001595059784449404
WCSMGP 3 sign 0.5
 
WCSMGP 6 MSE 0.000279434651019983
WCSMGP 6 sign 0.0
 
WCSMGP 9 MSE 0.00015051744776428677
WCSMGP 9 sign 1.0
 
CSMGP MSE 0.00035569193023547995
CSMGP sign 0.6
 
Benchmark 0.00037895642417424825
 
CU :
WCSMGP 3 MSE 0.00011097308379248716
WCSMGP 3 sign 0.5
 
WCSMGP 6 MSE 0.00010733939734564046
WCSMGP 6 sign 0.5
 
WCSMGP 9 MSE 0.00010747060878202319
WCSMGP 9 sign 0.5
 
CSMGP MSE 0.00016247960659256932
CSMGP sign 0.6
 
Benchmark 0.00016903027899388688
 
######
0
1
2
3
4
T+ 5  Total Count = 5
AL :
WCSMGP 3 MSE 0.0009096602300814993
WCSMGP 3 sign 0.4
 
WCSMGP 6 MSE 0.0011507070330026182
WCSMGP 6 sign 0.4
 
WCSMGP 9 MSE 0.0006586405644839032
WCSMGP 9 sign 0.6
 
CSMGP MSE 0.0011547449743375183
CSMGP sign 0.2
 
Benchmark 0.0006028308402164839
 
CU :
WCSMGP 3 MSE 0.0010005353124142857
WCSMGP 3 sign 0.6
 
WCSMGP 6 MSE 0.001403677098278422
WCSMGP 6 sign 0.4
 
WCSMGP 9 MSE 0.0013230283846496604
WCSMGP 9 sign 0.

In [0]:
Joint_Predictions = [['al_lme_prices_LD_1', 'cu_lme_prices_LD_1'],
                     ['al_lme_prices_LD_5', 'cu_lme_prices_LD_5'],
                     ['al_lme_prices_LD_22', 'cu_lme_prices_LD_22'] ]

horizons = [1, 5, 22]
complexity = [3,6,9]

for h, horizon in enumerate(horizons):
  Target_Variables = Joint_Predictions[h]
  WCSMGP_mse, WCSMGP_sign, CSMGP_mse, CSMGP_sign, BM_mse= Across_Metals(window = 100, 
                                                                     horizon = horizon, 
                                                                     Target_Variables = Target_Variables, 
                                                                     trials = 200,
                                                                     complexity = complexity)
  print('T+',horizon, ' Total Count = ' + str(BM_mse.shape[-1]) )
  for tv, TV in enumerate(Target_Variables):
    print(TV[0:2].upper(),':') #this is the metal
    
    for c, comp in enumerate(complexity):
      print('WCSMGP '+str(comp)+ ' MSE', np.mean(WCSMGP_mse[tv,c,:] ) )
      print('WCSMGP '+str(comp)+ ' sign', np.mean(WCSMGP_sign[tv,c,:]) )
      print(' ')
    
    print('CSMGP MSE', np.mean(CSMGP_mse[tv,:] ))
    print('CSMGP sign', np.mean(CSMGP_sign[tv,:] ))
    print(' ')
          
    print('Benchmark', np.mean(BM_mse[tv,:] ) )
    print(' ')
    
  print('######')

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17


/usr/local/lib/python3.6/dist-packages/numpy/core/fromnumeric.py:3118: RuntimeWarning: Mean of empty slice.
  out=out, **kwargs)
/usr/local/lib/python3.6/dist-packages/numpy/core/_methods.py:78: RuntimeWarning: invalid value encountered in true_divide
  ret, rcount, out=ret, casting='unsafe', subok=False)


18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
bad! 0
193
194
195
196
197
198
199
T+ 1  Total Count = 199
AL :
WCSMGP 3 MSE 0.0001875935431581085
WCSMGP 3 sign 0.5118110236220472
 
WCSMGP 6 MSE 0.00017773824046192805
WCSMGP 6 sign 0.5118110236220472
 
WCSMGP 9 MSE 0.0001878367139015991
WCSMGP 9 sign 0.5039370078740157
 
CSMGP MSE 0.09551374035457097
CSMGP sign 0.49246231155778897
 
Benchmark 0.00017086521490538962
 
CU :
WCSM

/usr/local/lib/python3.6/dist-packages/numpy/core/fromnumeric.py:3118: RuntimeWarning: Mean of empty slice.
  out=out, **kwargs)
/usr/local/lib/python3.6/dist-packages/numpy/core/_methods.py:78: RuntimeWarning: invalid value encountered in true_divide
  ret, rcount, out=ret, casting='unsafe', subok=False)


27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52


In [0]:
Joint_Predictions = [['al_lme_prices_LD_5', 'cu_lme_prices_LD_5'],
                     ['al_lme_prices_LD_22', 'cu_lme_prices_LD_22'] ]

horizons = [5, 22]
complexity = [3,6,9]

for h, horizon in enumerate(horizons):
  Target_Variables = Joint_Predictions[h]
  WCSMGP_mse, WCSMGP_sign, CSMGP_mse, CSMGP_sign, BM_mse= Across_Metals(window = 100, 
                                                                     horizon = horizon, 
                                                                     Target_Variables = Target_Variables, 
                                                                     trials = 200,
                                                                     complexity = complexity)
  print('T+',horizon, ' Total Count = ' + str(BM_mse.shape[-1]) )
  for tv, TV in enumerate(Target_Variables):
    print(TV[0:2].upper(),':') #this is the metal
    
    for c, comp in enumerate(complexity):
      print('WCSMGP '+str(comp)+ ' MSE', np.mean(WCSMGP_mse[tv,c,:] ) )
      print('WCSMGP '+str(comp)+ ' sign', np.mean(WCSMGP_sign[tv,c,:]) )
      print(' ')
    
    print('CSMGP MSE', np.mean(CSMGP_mse[tv,:] ))
    print('CSMGP sign', np.mean(CSMGP_sign[tv,:] ))
    print(' ')
          
    print('Benchmark', np.mean(BM_mse[tv,:] ) )
    print(' ')
    
  print('######')

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45


/usr/local/lib/python3.6/dist-packages/numpy/core/fromnumeric.py:3118: RuntimeWarning: Mean of empty slice.
  out=out, **kwargs)
/usr/local/lib/python3.6/dist-packages/numpy/core/_methods.py:78: RuntimeWarning: invalid value encountered in true_divide
  ret, rcount, out=ret, casting='unsafe', subok=False)


46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
T+ 5  Total Count = 200
AL :
WCSMGP 3 MSE 0.001030137195194811
WCSMGP 3 sign 0.5326633165829145
 
WCSMGP 6 MSE 0.0011592913231684267
WCSMGP 6 sign 0.5175879396984925
 
WCSMGP 9 MSE 0.0010995759333649513
WCSMGP 9 sign 0.5125628140703518
 
CSMGP MSE 24.619149988357893
CSMGP sign 0.535
 
Benchmark 0.0009132472079764842
 
CU :
WCSMGP 3 MSE 0.0017943426034877535
WCSMGP 3 sign 0.5527638190954773
 
WCSMGP 6 MSE 0.0016475469189774904
WCSMGP 6

/usr/local/lib/python3.6/dist-packages/numpy/core/fromnumeric.py:3118: RuntimeWarning: Mean of empty slice.
  out=out, **kwargs)
/usr/local/lib/python3.6/dist-packages/numpy/core/_methods.py:78: RuntimeWarning: invalid value encountered in true_divide
  ret, rcount, out=ret, casting='unsafe', subok=False)


3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132


In [0]:
Joint_Predictions = [['al_lme_prices_LD_22', 'cu_lme_prices_LD_22'] ]

horizons = [22]
complexity = [3,6,9]

for h, horizon in enumerate(horizons):
  Target_Variables = Joint_Predictions[h]
  WCSMGP_mse, WCSMGP_sign, CSMGP_mse, CSMGP_sign, BM_mse= Across_Metals(window = 100, 
                                                                     horizon = horizon, 
                                                                     Target_Variables = Target_Variables, 
                                                                     trials = 200,
                                                                     complexity = complexity)
  print('T+',horizon, ' Total Count = ' + str(BM_mse.shape[-1]) )
  for tv, TV in enumerate(Target_Variables):
    print(TV[0:2].upper(),':') #this is the metal
    
    for c, comp in enumerate(complexity):
      print('WCSMGP '+str(comp)+ ' MSE', np.mean(WCSMGP_mse[tv,c,:] ) )
      print('WCSMGP '+str(comp)+ ' sign', np.mean(WCSMGP_sign[tv,c,:]) )
      print(' ')
    
    print('CSMGP MSE', np.mean(CSMGP_mse[tv,:] ))
    print('CSMGP sign', np.mean(CSMGP_sign[tv,:] ))
    print(' ')
          
    print('Benchmark', np.mean(BM_mse[tv,:] ) )
    print(' ')
    
  print('######')

0
1
2
3
4
5
6
7


/usr/local/lib/python3.6/dist-packages/numpy/core/fromnumeric.py:3118: RuntimeWarning: Mean of empty slice.
  out=out, **kwargs)
/usr/local/lib/python3.6/dist-packages/numpy/core/_methods.py:78: RuntimeWarning: invalid value encountered in true_divide
  ret, rcount, out=ret, casting='unsafe', subok=False)


8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
T+ 22  Total Count = 200
AL :
WCSMGP 3 MSE 0.0071076724819010145
WCSMGP 3 sign 0.5053191489361702
 
WCSMGP 6 MSE 0.007320811415677979
WCSMGP 6 sign 0.5585106382978723
 
WCSMGP 9 MSE 0.007405690446752658
WCSMGP 9 sign 0.5212765957446809
 
CSMGP MSE 58.055927689298024
CSMGP sign 0.485
 
Benchmark 0.0039948062964511874
 
CU :
W

# Confidence Intervals

In [0]:
def Across_Metals_CI(window, horizon, Target_Variables, trials, complexity):

  # metal is either 'al' or 'cu'
  # window is the length of data before prediction
  # Horizons is a list of numbers (e.g. [1,5,22])
  # Target_Variables is a list of strings 
  #      (e.g. ['al_lme_prices_LD_1', 'cu_lme_prices_LD_1'])
  # kernel is the kernel that we will use 
  # trials (self explanatory)
  # n_test (number of test points per trial)
  
  I = len(Target_Variables)
  
  # this
  WCSMGP_interval = np.zeros((I,trials))
  WCSMGP_included = np.zeros((I,trials))

  BM_mse = np.zeros((I,trials))
  
  # this contains the list of dataframes 
  DF_list = []
  for TV_index,TV in enumerate(Target_Variables):
    # horizon and target variable
    DF = feature_generation(TV[0:2], time_pred = [1,5,22])
    DF = feature_extraction(DF, TV, horizon)
    DF_list.append(DF)
  
  # now join the DFs in DF_list, and remove duplicate columns
  DF = pd.concat(DF_list, axis = 1, sort = True).dropna()
  DF = DF.loc[:,~DF.columns.duplicated()]
  Y = DF.loc[:,Target_Variables]
  X = DF.drop(Target_Variables, axis = 1)

#   print((Y**2).mean())
  
  no_conv = []
  
  Chosen = np.random.choice(np.arange(len(Y) - 200),trials, replace = False)
  
  for i in range(trials):
    print(i)
    START = Chosen[i]
    END = START + window
    X_train = torch.tensor( X.iloc[START:END, :].values , dtype = torch.float32)
    X_test = torch.tensor( X.iloc[[(END+horizon-1)], :].values , dtype = torch.float32)
    Y_train = torch.tensor( Y.iloc[START:END, :].values.squeeze() , dtype = torch.float32)
    Y_test = torch.tensor( Y.iloc[[(END+horizon-1)], :].values.squeeze() , dtype = torch.float32)
    
    try:
      WCSMGP_interval[:,i], WCSMGP_included[:,i] = WCSMGP_simulation_CI(complexity = complexity, 
                                                              X = X_train, Y = Y_train, 
                                                              x = X_test, y = Y_test)
      skip = 0
    except:
      print('bad!', i)
      skip = 1
      
    if skip == 1:
      skip = 0
      no_conv.append(i)
      continue
    
  
  return_list = [WCSMGP_interval, WCSMGP_included]
  
  return_list = [np.delete(elt, no_conv, -1) for elt in return_list]
  return_list = [np.delete(elt, np.where(np.isnan(elt))[-1], -1) for elt in return_list]
      
  return return_list#WCSMGP_mse, WCSMGP_sign, CSMGP_mse, CSMGP_sign, BM_mse


def WCSMGP_simulation_CI(complexity, X, Y, x, y):
    #### setup ##########################################
    llh = []
    model_MGP = MGP(X = X, Y= Y, kernel = RBF, rank = 2, ARD = False)
    model_SIGP = SIGP(X = X, Y= Y, kernel = RBF, ARD = False)

    Warp = Warp_MT(tasks = 2, complexity=complexity)
    LLH = WCSMGP_Likelihood()

    alpha = torch.nn.Parameter(torch.tensor(0.))
    optimizer1 = torch.optim.Adam(params = model_MGP.get_params() + \
                                 model_SIGP.get_params() + \
                                 Warp.get_params() + \
                                 [alpha], lr = 0.1)

    #### training ##########################################
    for i in range(1000):

        _alpha = torch.sigmoid(alpha)

        K = _alpha*model_SIGP.get_cov() \
            + (1-_alpha)*model_MGP.get_cov()
        loss = LLH.forward(K, Y, Warp)

        optimizer1.zero_grad()
        loss.backward()
        optimizer1.step()
        
        llh.append(loss.squeeze().detach().numpy())
    
    tol_flag, interval, included = WCSMGP_predict_CI(X = model_SIGP.X, Y = Y, y= y,
                                                  Models=[model_SIGP, model_MGP], 
                                                  alphas=[_alpha, 1- _alpha],
                                                  Warp= Warp,
                                                  x=x)
      
    
    return interval.detach().numpy(), included.detach().numpy()
  
def WCSMGP_predict_CI(X, Y, x, y, Models, Warp, alphas = None):
    
    # Y should be a n x task matrix
    _Z = Warp.f(Y).squeeze().t().contiguous().reshape(1,-1).t().contiguous()
    
    if alphas == None:
        alphas = torch.ones(len(Models), dtype = torch.float32)
    K_XX = reduce(lambda x, y: x + y, 
                  [alpha*model.get_cov() for alpha,model in zip(alphas, Models)] )
    K_xX = reduce(lambda x, y: x + y, 
                  [alpha*model.get_K_xX(x) for alpha,model in zip(alphas, Models)] )
    K_xx = reduce(lambda x, y: x + y, 
                  [alpha*model.get_K_xx(x) for alpha,model in zip(alphas, Models)] )
    pred_mean = K_xX.mm(torch.inverse(K_XX)).mm(_Z).squeeze()
    
    pred_cov = K_xx - K_xX.mm(torch.inverse(K_XX)).mm(K_xX.t())
    pred_cov = torch.diag(pred_cov)**0.5
    
    upper_latent = pred_mean + 1.96 * pred_cov
    lower_latent = pred_mean - 1.96 * pred_cov
    tol_flag1, upper_obs = Warp.inv(0.001, upper_latent)
    tol_flag2, lower_obs = Warp.inv(0.001, lower_latent)
    
    pred_mean = pred_mean.reshape(2,-1).t().contiguous()
    
#     tol_flag3, pred_mean = Warp.inv(Z = pred_mean,eps=0.0001)
    
    interval = upper_obs - lower_obs
    included = (upper_obs > y) * (lower_obs < y)
    
    
    return(tol_flag1*tol_flag2, interval, included)

In [58]:
Joint_Predictions = [['al_lme_prices_LD_1', 'cu_lme_prices_LD_1'],
                     ['al_lme_prices_LD_5', 'cu_lme_prices_LD_5'],
                     ['al_lme_prices_LD_22', 'cu_lme_prices_LD_22'] ]
                                                                                                                                                                                                                                                                                                                          
horizons = [1, 5, 22]
complexity = 6

for h, horizon in enumerate(horizons):
  Target_Variables = Joint_Predictions[h]
  WCSMGP_interval, WCSMGP_included = Across_Metals_CI(window = 100, 
                                                   horizon = horizon, 
                                                   Target_Variables = Target_Variables, 
                                                   trials = 100,
                                                   complexity = complexity)
  print('T+',horizon, ' Total Count = ' + str(WCSMGP_interval.shape[-1]) )
  for tv, TV in enumerate(Target_Variables):
    print(TV[0:2].upper(),':') #this is the metal

    print('WCSMGP interval', np.mean(WCSMGP_interval[tv,:] ) )
    print('WCSMGP included', np.mean(WCSMGP_included[tv,:]) )
    print(' ')

    
  print('######')

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
T+ 1  Total Count = 100
AL :
WCSMGP interval 0.049015620816498996
WCSMGP included 0.92
 
CU :
WCSMGP interval 0.05597432874143124
WCSMGP included 0.88
 
######
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
T+ 5  Total Count = 100
AL :
WCSMGP interval 0.1190477615967393
WCSMGP included 0.88
 
CU :
WCSMGP interval 0.13626656141132115
WCSMGP included 0.91
 
######
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
3

In [7]:
Joint_Predictions = [['al_lme_prices_LD_1', 'cu_lme_prices_LD_1'],
                     ['al_lme_prices_LD_5', 'cu_lme_prices_LD_5'],
                     ['al_lme_prices_LD_22', 'cu_lme_prices_LD_22'] ]
                                                                                                                                                                                                                                                                                                                          
horizons = [1, 5, 22]
complexity = 6

for h, horizon in enumerate(horizons):
  Target_Variables = Joint_Predictions[h]
  WCSMGP_interval, WCSMGP_included = Across_Metals_CI(window = 100, 
                                                   horizon = horizon, 
                                                   Target_Variables = Target_Variables, 
                                                   trials = 300,
                                                   complexity = complexity)
  print('T+',horizon, ' Total Count = ' + str(WCSMGP_interval.shape[-1]) )
  for tv, TV in enumerate(Target_Variables):
    print(TV[0:2].upper(),':') #this is the metal

    print('WCSMGP interval', np.mean(WCSMGP_interval[tv,:] ) )
    print('WCSMGP included', np.mean(WCSMGP_included[tv,:]) )
    print(' ')

    
  print('######')

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
27

In [37]:
ra = torch.tensor([-0.5039, -0.7520])
print(ra)
Warp.inv(0.001, ra)

tensor([-0.5039, -0.7520])


(tensor([1], dtype=torch.uint8),
 tensor([-0.0578,  0.1118], grad_fn=<AsStridedBackward>))

In [38]:
Warp.f(Warp.inv(0.001, ra)[1])

tensor([[[-0.5039, -0.7520]]], grad_fn=<AddBackward0>)